In [1]:
import pandas as pd
import time

In [2]:
import cv2
import numpy as np
from keras.models import load_model
from statistics import mode
from utils.datasets import get_labels
from utils.inference import detect_faces
from utils.inference import draw_text
from utils.inference import draw_bounding_box
from utils.inference import apply_offsets
from utils.inference import load_detection_model
from utils.preprocessor import preprocess_input

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
%matplotlib inline
# import scipy.io
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import dlib
from wide_resnet import WideResNet

# load model and weights
img_size = 64
model = WideResNet(img_size, depth=16, k=8)()
model.load_weights(os.path.join("pretrained_models", "weights.18-4.06.hdf5"))

In [14]:
a = 21
g = 'male'

In [4]:
USE_WEBCAM = True # If false, loads video file source

# parameters for loading data and images
emotion_model_path = './models/emotion_model.hdf5'
emotion_labels = get_labels('fer2013')

In [5]:
# hyper-parameters for bounding boxes shape
frame_window = 10
emotion_offsets = (20, 40)

In [6]:
# loading models
face_cascade = cv2.CascadeClassifier('./models/haarcascade_frontalface_default.xml')
emotion_classifier = load_model(emotion_model_path)

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\keras\models.py:291: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [7]:
# getting input model shapes for inference
emotion_target_size = emotion_classifier.input_shape[1:3]

# starting lists for calculating modes
emotion_window = []

In [ ]:
res_df = pd.DataFrame(columns=['time', 'reaction', 'reaction_tense', 'gender', 'age'])

filename = './test_recordings/test_male_4.mp4'
filename_result = './result_emotions/male_4.csv'

# starting video streaming

cv2.namedWindow('window_frame')
video_capture = cv2.VideoCapture(0)

# Select video or webcam feed
cap = None
if (USE_WEBCAM == True):
    cap = cv2.VideoCapture(0) # Webcam source
else:
    cap = cv2.VideoCapture(filename) # Video file source

old_gender = ''
old_age = 0
i = 0
start_time = time.time()
while cap.isOpened(): # True:
    ret, bgr_image = cap.read()

    #bgr_image = video_capture.read()[1]

    gray_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)
    rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
    
    num_rows, num_cols = gray_image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2), 90, 1)
#     gray_image = cv2.warpAffine(gray_image, rotation_matrix, (num_cols, num_rows))
#     rgb_image = cv2.warpAffine(rgb_image, rotation_matrix, (num_cols, num_rows))

    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5,
        minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    for face_coordinates in faces:

        x1, x2, y1, y2 = apply_offsets(face_coordinates, emotion_offsets)
        gray_face = gray_image[y1:y2, x1:x2]
        rgb_face = rgb_image[y1:y2, x1:x2]
        try:
            gray_face = cv2.resize(gray_face, (emotion_target_size))
            rgb_face = cv2.resize(rgb_face, (emotion_target_size))
        except:
            continue

        gray_face = preprocess_input(gray_face, False)
        gray_face = np.expand_dims(gray_face, 0)
        gray_face = np.expand_dims(gray_face, -1)
        rgb_face = preprocess_input(rgb_face, False)
        rgb_face = np.expand_dims(rgb_face, 0)
        rgb_face = np.expand_dims(rgb_face, -1)
        emotion_prediction = emotion_classifier.predict(gray_face)
        emotion_probability = np.max(emotion_prediction)
        emotion_label_arg = np.argmax(emotion_prediction)
        emotion_text = emotion_labels[emotion_label_arg]

        if emotion_text == 'angry':
            color = emotion_probability * np.asarray((255, 0, 0))
        elif emotion_text == 'sad':
            color = emotion_probability * np.asarray((0, 0, 255))
        elif emotion_text == 'happy':
            color = emotion_probability * np.asarray((255, 255, 0))
        elif emotion_text == 'surprise':
            color = emotion_probability * np.asarray((0, 255, 255))
        else:
            color = emotion_probability * np.asarray((0, 255, 0))
        
        if i % 10 == 0:
            age_gender = model.predict(rgb_face.reshape((1, 64, 64, 3)))
            predicted_genders = age_gender[0][0]
            ages = np.arange(0, 101).reshape(101, 1)
            predicted_ages = int(age_gender[1].dot(ages).flatten()[0])
            
            if predicted_genders[0] > 0.5:
                predicted_genders = 'male'
            else:
                predicted_genders = 'female'
                
            old_gender = predicted_genders
            old_age = predicted_ages
        else:
            predicted_genders = old_gender
            predicted_ages = old_age

        if a != 0:
            predicted_ages = a
        if g != '':
            predicted_genders = g
            
        res_df.loc[i, :] = [time.time() - start_time, emotion_text, emotion_probability, predicted_genders, predicted_ages]
        i += 1
                            
        emotion_window.append(emotion_text + ', ' + predicted_genders + ', ' + str(predicted_ages))
        
        if len(emotion_window) > frame_window:
            emotion_window.pop(0)
        try:
            emotion_mode = mode(emotion_window)
        except:
            continue
            
        color = color.astype(int)
        color = color.tolist()

        draw_bounding_box(face_coordinates, rgb_image, color)
        draw_text(face_coordinates, rgb_image, emotion_mode,
                  color, 0, -45, 1, 1)

    bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)
    cv2.imshow('window_frame', bgr_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
# res_df.to_csv(filename_result, index = False)

In [9]:
# res_df.to_csv(filename_result, index = False)